### 0. Directory setting

In [3]:
import os
import sys

notebook_dir = os.getcwd()

project_dir = notebook_dir
if os.path.basename(notebook_dir) == 'notebooks':
    project_dir = os.path.dirname(notebook_dir)
    os.chdir(project_dir)

if project_dir not in sys.path:
    sys.path.insert(0, project_dir)
    



### 1. Imports

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import logging

# Import custom modules
import sys
sys.path.append('./')
from utils.data_loader import load_data, load_anomaly_labels, normalize_data, create_sequences
from utils.data_loader import create_labels_array, split_data, prepare_data_loaders
from utils.model_utils import LSTMAutoencoder, GRUAutoencoder, TransformerEncoder, train_model
from utils.model_utils import get_reconstruction_errors, EnsembleModel, save_trained_models
from utils.evaluation import evaluate_threshold, AnomalyInterpreter, visualize_results
from utils.evaluation import plot_roc_curves, plot_precision_recall_curves, create_results_summary, print_results_table


### 2. Initial Configuration



In [ ]:

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(f"Usando dispositivo: {device}")


### 3. Data Loading and Preprocessing

In [ ]:

# Load and prepare data
filepath = 'data/machine_temperature_system_failure.csv'
labels_path = 'data/known_labels_v1.0.json'
windows_path = 'data/combined_windows.json'

df = load_data(filepath)
labels_dict, windows_dict = load_anomaly_labels(labels_path, windows_path)

filename = 'realKnownCause/machine_temperature_system_failure.csv'
anomaly_timestamps = labels_dict[filename]
anomaly_windows = windows_dict[filename]

data_scaled, scaler = normalize_data(df[['value']])


In [ ]:

sequence_length = 150
step = 10

X = create_sequences(data_scaled, sequence_length, step)
y = create_labels_array(df, anomaly_timestamps, anomaly_windows, sequence_length, step)

X_train, y_train, X_val, y_val, X_test, y_test = split_data(X, y)

batch_size = 64
train_loader, val_loader, test_loader = prepare_data_loaders(X_train, X_val, X_test, batch_size)


### 4. Construct Models

In [ ]:

# Train models
lstm_autoencoder = LSTMAutoencoder(seq_len=sequence_length, n_features=1, embedding_dim=64, dropout=0.2)
gru_autoencoder = GRUAutoencoder(seq_len=sequence_length, n_features=1, embedding_dim=64, dropout=0.2)
transformer_model = TransformerEncoder(seq_len=sequence_length, n_features=1, d_model=48, nhead=4,
                                       dropout=0.3, num_layers=2)

lstm_history, lstm_model = train_model(
    lstm_autoencoder, train_loader, val_loader, n_epochs=50, learning_rate=1e-3,
    device=device, patience=7, min_delta=0.0001, weight_decay=1e-5
)

gru_history, gru_model = train_model(
    gru_autoencoder, train_loader, val_loader, n_epochs=50, learning_rate=1e-3,
    device=device, patience=7, min_delta=0.0001, weight_decay=1e-5
)


In [ ]:

# Pre-train Transformer model on normal data only
normal_mask = y_train == 0
X_normal = torch.FloatTensor(X_train[normal_mask])
normal_dataset = torch.utils.data.TensorDataset(X_normal)
normal_loader = torch.utils.data.DataLoader(normal_dataset, batch_size=64, shuffle=True)

transformer_model = transformer_model.to(device)
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = torch.nn.MSELoss()

for epoch in range(20):
    transformer_model.train()
    total_loss = 0
    for batch in normal_loader:
        seq_true = batch[0].to(device)
        optimizer.zero_grad()
        seq_pred = transformer_model(seq_true)
        loss = criterion(seq_pred, seq_true)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    logger.info(f"Transformer Pretraining Epoch {epoch+1}/20, Loss: {total_loss/len(normal_loader):.5f}")

transformer_history, transformer_model = train_model(
    transformer_model, train_loader, val_loader, n_epochs=50, learning_rate=1e-3,
    device=device, patience=7, min_delta=0.0001, weight_decay=1e-5
)


In [ ]:

# Get reconstruction errors
lstm_errors, lstm_orig, lstm_recon = get_reconstruction_errors(lstm_model, test_loader, device)
gru_errors, gru_orig, gru_recon = get_reconstruction_errors(gru_model, test_loader, device)
transformer_errors, transformer_orig, transformer_recon = get_reconstruction_errors(transformer_model, test_loader, device)

val_lstm_errors, _, _ = get_reconstruction_errors(lstm_model, val_loader, device)
val_gru_errors, _, _ = get_reconstruction_errors(gru_model, val_loader, device)
val_transformer_errors, _, _ = get_reconstruction_errors(transformer_model, val_loader, device)


In [ ]:

# Create ensemble model
ensemble = EnsembleModel(
    models=[lstm_model, gru_model, transformer_model],
    names=["LSTM", "GRU", "Transformer"]
)

ensemble.optimize_weights(
    [val_lstm_errors, val_gru_errors, val_transformer_errors],
    y_val
)

ensemble_errors, individual_errors = ensemble.get_weighted_errors(
    [test_loader, test_loader, test_loader],
    device
)

val_ensemble_errors, _ = ensemble.get_weighted_errors(
    [val_loader, val_loader, val_loader],
    device
)


### 5. Evaluate Models

In [ ]:

# Evaluate models
lstm_results = evaluate_threshold(lstm_errors, y_test, val_lstm_errors, factor=1.5)
gru_results = evaluate_threshold(gru_errors, y_test, val_gru_errors, factor=1.5)
transformer_results = evaluate_threshold(transformer_errors, y_test, val_transformer_errors, factor=1.5)
ensemble_results = evaluate_threshold(ensemble_errors, y_test, val_ensemble_errors, factor=1.5)

# Print results
print("\n=== Results with Adaptive Threshold (factor 1.5) ===")
results_summary = {
    'lstm': lstm_results,
    'gru': gru_results,
    'transformer': transformer_results,
    'ensemble': ensemble_results
}
print_results_table(create_results_summary({
    'results': results_summary,
    'thresholds': {
        'lstm': lstm_results['threshold'],
        'gru': gru_results['threshold'],
        'transformer': transformer_results['threshold'],
        'ensemble': ensemble_results['threshold']
    }
}))


### 6. Interpret and Analyze Anomalies

In [ ]:

# Create interpreter
timestamps = df.index[sequence_length:][::step][-len(ensemble_errors):]
interpreter = AnomalyInterpreter(
    model=None,
    test_data=lstm_orig,
    reconstructed_data=lstm_recon,
    errors=ensemble_errors,
    thresholds=ensemble_results['threshold'],
    timestamps=timestamps
)

# Analyze top anomalies
top_anomalies = interpreter.analyze_top_anomalies(top_k=5)
print("\n=== Top 5 Detected Anomalies ===")
for i, anomaly in enumerate(top_anomalies):
    print(f"Anomaly #{i+1}:")
    print(f"  Index: {anomaly['index']}")
    if 'timestamp' in anomaly:
        print(f"  Timestamp: {anomaly['timestamp']}")
    print(f"  Error: {anomaly['error']:.6f}")
    print(f"  Threshold: {anomaly['threshold']:.6f}")
    print(f"  Error/Threshold Ratio: {anomaly['error_ratio']:.4f}")


### 7. Prepare results dictionary

In [ ]:

# Prepare results dictionary
results_dict = {
    'models': {
        'lstm': lstm_model,
        'gru': gru_model,
        'transformer': transformer_model,
        'ensemble': ensemble
    },
    'errors': {
        'lstm': lstm_errors,
        'gru': gru_errors,
        'transformer': transformer_errors,
        'ensemble': ensemble_errors
    },
    'thresholds': {
        'lstm': lstm_results['threshold'],
        'gru': gru_results['threshold'],
        'transformer': transformer_results['threshold'],
        'ensemble': ensemble_results['threshold']
    },
    'results': {
        'lstm': lstm_results,
        'gru': gru_results,
        'transformer': transformer_results,
        'ensemble': ensemble_results
    },
    'data': {
        'X_test': X_test,
        'y_test': y_test,
        'lstm_orig': lstm_orig,
        'lstm_recon': lstm_recon,
        'timestamps': timestamps
    },
    'interpreter': interpreter
}

# Visualize results
visualize_results(results_dict)
plot_roc_curves(results_dict)
plot_precision_recall_curves(results_dict)


### 8. Save Trained Models

In [ ]:

# Save models
config = {
    "sequence_length": sequence_length,
    "embedding_dim": 64
}

save_trained_models(
    models_dict={
        "lstm": lstm_model,
        "gru": gru_model,
        "transformer": transformer_model
    },
    results_dict={
        "lstm": lstm_results,
        "gru": gru_results,
        "transformer": transformer_results,
        "ensemble": ensemble_results
    },
    config=config,
    base_path="models/autoencoder/"
)